In [1]:
if 'google.colab' in str(get_ipython()):
  # install packages required for this tutorial
  !pip install tensorflow==2.3.1
  !pip install tensorflow_quantum==0.4.0
  !pip install quple==0.7.8

# Tutorial-04 Encoding Function

In this tutorial, you will learn how to:

- use various encoding functions for data encoding
- create your own encoding functions

# Encoding Function

An encoding function $\phi(\mathbf{x})$ specifies how input features $\mathbf{x}$ are encoded into the parameters of a unitary gate operation in an encoding circuit. Usually, the parameters involved in a unitary gate operation are the rotation angles about some axis of the single qubit or multi-qubit Bloch sphere depending on the number of qubits the gate operation is acting on. Therefore, for input features of dimension $n$, the encoding function is a map $f: \mathbb{R}^n \rightarrow \mathbb{R}$. It is natural to restrict the range of the encoding function to be within $(0, 2\pi)$ or $(-\pi, \pi)$ to correspond to the effective range of an angle of rotation. 
	
In Quple, encoding circuits from the `EncodingCircuit` class (see next tutorial) will have its gate operations parameterized by the expressions of the form $\pi\phi(\mathbf{x})$ with the $\pi$ factor extracted out by default which restricts the range of $\phi(\mathbf{x})$ to be within $[0, 2]$ or $[-1, 1]$. One can also remove this $\pi$ factor by specifying the `parameter_scale` argument when initializing the `EncodingCircuit` instance. There are a number of encoding functions that are implemented in Quple which also put a restriction on the value of each input feature to be within $[-1, +1]$ to make sure the encoding functions will map input features of arbitrary dimension to a value of the required range.  Users can also create their own encoding function as  long as it takes an array like input $\mathbf{x}$ and output a number that is within the required range. 

Some of the available encoding functions are

| Encoding Function   |    Formula  ($n=1$)   |  <div style="width:500px">     Formula($n>1$)    </div> | Domain  |  <div style="width:500px">  Range  </div> |
| :-----------------: |:-------------:| :-------------:| :-------:| :------:|
| self_product      | $x_0$    | $\prod_{i=0}^n x_i$ | [-1, +1] | [-1, +1] |
|cosine_product | $x_0$ |$\prod_{i=0}^n (\cos(\pi(x_i+1)/2))$ | [-1, +1] | [1, +1] |	
|distance_measure |$x_0$ |$\prod_{i<j} (x_i-x_j)/2^{\text{pairs}}$  | [-1, +1] | [-1, +1]  |
|one_norm_distance |$x_0$ | $\sum_{i<j} |x_i-x_j|/\text{pairs}$ | [-1, +1] | [0, +2] |							
|two_norm_distance |$x_0$ | $[\sum_{i<j} (x_i-x_j)^2/{\text{pairs}}]^{1/2}$ | [-1, +1] | [0, +2]  |	
|arithmetic_mean | $x_0$ |	$\sum_{i=0}^n x_i/n$ | [-1, +1] | [-1, +1]  |								
|second_moment | $x_0$ |	$[\sum_{i=0}^n (x_i+1)^2/n]^{1/2}$ | [-1, +1] | [-1, +1]  |								
|cube_sum | 	$x_0$ |$\sum_{i=0}^n x_i^3/n$ | [-1, +1] | [-1, +1]  |								
|exponential_square_sum | $x_0$ |	$2\exp[(\sum_{i=0}^n x_i^2/n)-1]$ | [-1, +1] | [$2\exp(-1)$, +2] |								
|exponential_cube_sum | $x_0$ |	$2\exp[(\sum_{i=0}^n x_i^3/n)-1]$ | [-1, +1] | [$2\exp(-2)$, +2] |
|polynomial | $x_0^d$ |	$[\sum_{i=0}^n x_i/n]^d$ | [-1, +1] | [-1, +1] |

In [2]:
from quple.data_encoding.encoding_maps import (self_product, cosine_product, modified_cosine_product, 
distance_measure, one_norm_distance, two_norm_distance, arithmetic_mean, second_moment, cube_sum, 
exponential_cube_sum, exponential_square_sum, polynomial)

In [3]:
# Prepare the parameter symbols
import sympy as sp
x = sp.symarray('x', 5)

In [4]:
# this version of cosine product is without rescaling the range of angles
cosine_product(x)

cos(x_0)*cos(x_1)*cos(x_2)*cos(x_3)*cos(x_4)

In [5]:
# this version of cosine product will rescale the range of angles
modified_cosine_product(x)

cos(pi*(x_0/2 + 1/2))*cos(pi*(x_1/2 + 1/2))*cos(pi*(x_2/2 + 1/2))*cos(pi*(x_3/2 + 1/2))*cos(pi*(x_4/2 + 1/2))

In [6]:
distance_measure(x)

(-x_0 + x_4)*(x_0 - x_1)*(x_1 - x_2)*(x_2 - x_3)*(x_3 - x_4)/32

In [7]:
one_norm_distance(x)

Abs(x_0 - x_1)/5 + Abs(x_0 - x_4)/5 + Abs(x_1 - x_2)/5 + Abs(x_2 - x_3)/5 + Abs(x_3 - x_4)/5

In [8]:
two_norm_distance(x)

((-x_0 + x_4)**2/5 + (x_0 - x_1)**2/5 + (x_1 - x_2)**2/5 + (x_2 - x_3)**2/5 + (x_3 - x_4)**2/5)**0.5

In [9]:
arithmetic_mean(x)

x_0/5 + x_1/5 + x_2/5 + x_3/5 + x_4/5

In [10]:
second_moment(x)

((x_0 + 1)**2/5 + (x_1 + 1)**2/5 + (x_2 + 1)**2/5 + (x_3 + 1)**2/5 + (x_4 + 1)**2/5)**0.5

In [11]:
cube_sum(x)

x_0**3/5 + x_1**3/5 + x_2**3/5 + x_3**3/5 + x_4**3/5

In [12]:
exponential_square_sum(x)

2*exp(x_0**2/5 + x_1**2/5 + x_2**2/5 + x_3**2/5 + x_4**2/5 - 1)

In [13]:
exponential_cube_sum(x)

2*exp(x_0**3/5 + x_1**3/5 + x_2**3/5 + x_3**3/5 + x_4**3/5 - 1)

In [14]:
# for the "polynomial" encoding function, one can specify a degree parameter, d, to raise 
# the weighted sum of input values by the degree
polynomial(degree=2)(x)

(x_0/5 + x_1/5 + x_2/5 + x_3/5 + x_4/5)**2/pi

In [15]:
polynomial(degree=5, scale_factor=1)(x)

(x_0/5 + x_1/5 + x_2/5 + x_3/5 + x_4/5)**5

In [16]:
# now try out the encoding function on an EncodingCircuit instance
from quple.data_encoding import GeneralPauliEncoding
# Construct an encoding circuit with GeneralPauliEncoding using the Paulis 'Z' and 'ZZ' for encoding data of feature dimension 3 using the cube sum encoding function. 
encoding_circuit = GeneralPauliEncoding(feature_dimension=3, paulis=['Z', 'ZZ'],
                                        encoding_map=polynomial(degree=3), entangle_strategy='linear', copies=1)
encoding_circuit

(0, 0): ───H───Rz(pi*<x_0**3/pi>)───@────────────────────────────────────@────────────────────────────────────────────
                                    │                                    │
(0, 1): ───H───Rz(pi*<x_1**3/pi>)───X───Rz(pi*<(x_0/2 + x_1/2)**3/pi>)───X───@────────────────────────────────────@───
                                                                             │                                    │
(0, 2): ───H───Rz(pi*<x_2**3/pi>)────────────────────────────────────────────X───Rz(pi*<(x_1/2 + x_2/2)**3/pi>)───X───

In [17]:
import cirq
encoding_circuit = GeneralPauliEncoding(feature_dimension=4, paulis=['Z', 'ZZ', 'ZZZ'],
                                        encoding_map='cosine_product', entangle_strategy='linear', copies=1,
                                        insert_strategy=cirq.circuits.InsertStrategy.NEW)
encoding_circuit

(0, 0): ───H───────────────Rz(pi*x_0)────────────────@──────────────────────────────────────────────────────@────────────────────────────────────────@──────────────────────────────────────────────────────────────────────────────────@─────────────────────────────────────────────────────────
                                                     │                                                      │                                        │                                                                                  │
(0, 1): ───────H────────────────────────Rz(pi*x_1)───X─────────────────────────Rz(pi*<cos(x_0)*cos(x_1)>)───X───@────────────────────────────────@───X────────────────────────────────────@─────────────────────────────────────────@───X───@─────────────────────────────────────────────────@───
                                                                                                                │                                │                                        │                                         │       │                                                 │
(0, 2): ───────────H─────────────────────────────────Rz(pi*x_2)─────────────────────────────────────────────────X───Rz(pi*<cos(x_1)*cos(x_2)>)───X───@────────────────────────────────@───X───Rz(pi*<cos(x_0)*cos(x_1)*cos(x_2)>)───X───────X───@─────────────────────────────────────────@───X───
                                                                                                                                                     │                                │                                                         │                                         │
(0, 3): ───────────────H──────────────────────────────────────────Rz(pi*x_3)─────────────────────────────────────────────────────────────────────────X───Rz(pi*<cos(x_2)*cos(x_3)>)───X─────────────────────────────────────────────────────────X───Rz(pi*<cos(x_1)*cos(x_2)*cos(x_3)>)───X───────

## Create customized encoding function

In [18]:
def naive_encoding(x) -> float:
    """
    Function: (x_0 + x_1 - x_2 + x_3 ...)/n
    Domain: (-1, +1)
    Range: (-1, +1)

    Args:
        x: data

    Returns:
        float: the mapped value
    """
    coeff = 0
    for i, x_ in enumerate(x):
        if i % 2 == 0:
            coeff += x_
        else:
            coeff -= x_
    coeff /= len(x)
    return coeff

In [19]:
naive_encoding(x)

x_0/5 - x_1/5 + x_2/5 - x_3/5 + x_4/5

In [20]:
encoding_circuit = GeneralPauliEncoding(feature_dimension=5, paulis=['Z','ZZ', 'ZZZ'],
                                        encoding_map=naive_encoding, entangle_strategy='linear', copies=1)
encoding_circuit

(0, 0): ───H───Rz(pi*x_0)───@────────────────────────────@────────────────────────────────────@──────────────────────────────────────────────────────────────────────────────────────────────────@─────────────────────────────────────────────────────────────────────────────────────────────────────
                            │                            │                                    │                                                                                                  │
(0, 1): ───H───Rz(pi*x_1)───X───Rz(pi*<x_0/2 - x_1/2>)───X───@────────────────────────────@───X────────────────────────────────@─────────────────────────────────────────────────────────────@───X───@────────────────────────────────────────────@────────────────────────────────────────────────────
                                                             │                            │                                    │                                                             │       │                                            │
(0, 2): ───H───Rz(pi*x_2)────────────────────────────────────X───Rz(pi*<x_1/2 - x_2/2>)───X───@────────────────────────────@───X────────────────────────────Rz(pi*<x_0/3 - x_1/3 + x_2/3>)───X───────X───@────────────────────────────────────@───X───@────────────────────────────────────────────@───
                                                                                              │                            │                                                                             │                                    │       │                                            │
(0, 3): ───H───Rz(pi*x_3)─────────────────────────────────────────────────────────────────────X───Rz(pi*<x_2/2 - x_3/2>)───X───@────────────────────────────@────────────────────────────────────────────X───Rz(pi*<x_1/3 - x_2/3 + x_3/3>)───X───────X───@────────────────────────────────────@───X───
                                                                                                                               │                            │                                                                                             │                                    │
(0, 4): ───H───Rz(pi*x_4)──────────────────────────────────────────────────────────────────────────────────────────────────────X───Rz(pi*<x_3/2 - x_4/2>)───X─────────────────────────────────────────────────────────────────────────────────────────────X───Rz(pi*<x_2/3 - x_3/3 + x_4/3>)───X───────